# Box pendulum
- This is modelling for the rotary pendulum
- $m_P$ is the mass of the pendulum
- $m_A$ is the mass of the Arm
- $\theta$ is the angle made by the arm from the principal axis
- $\alpha$ is the angle made by the pendulum from the z axis of the arm

In [1]:
# Rotation matrices
import sympy as sym
import numpy as np
from IPython.display import display #for pretty printing

# Define the rotation matrices as functions, counterclockwise rotation
# axes system is defined similar to the one used for making the 3d plots
def rotX(th):
    return sym.Matrix([ [1,           0,            0],
                        [0, sym.cos(th), -sym.sin(th)],
                        [0, sym.sin(th),  sym.cos(th)]])

def rotY(th):
    return sym.Matrix([ [ sym.cos(th), 0, sym.sin(th)],
                        [ 0,           1,           0],
                        [-sym.sin(th), 0, sym.cos(th)]])

def rotZ(th):
    return sym.Matrix([ [sym.cos(th), -sym.sin(th), 0],
                        [sym.sin(th),  sym.cos(th), 0],
                        [0,        0,               1]])

In [ ]:
# create symbolic variables

# system parameters
X0,Y0, Z0 = sym.symbols(['X0','Y0', 'Z0']) # fixed position of Arm
g = sym.symbols('g')

mA, La, InA = sym.symbols(['m_A', 'l_A', 'I_A']) # Arm
mP, Lp, InP = sym.symbols(['m_P', 'l_P', 'I_P']) # Pen

# generalized coordinates
th, dth, ddth       = sym.symbols(['\\theta', '\\dot{\\theta}', '\\ddot{\\theta}'])
alph, dalph, ddalph = sym.symbols(['\\alpha', '\\dot{\\alpha}', '\\ddot{\\alpha}'])

q   = sym.Matrix([  [th],  [alph]]) #group into matrices
dq  = sym.Matrix([ [dth], [dalph]])
ddq = sym.Matrix([[ddth],[ddalph]])

display(ddq) #display prints it as cool latex stuff

# Rotation Matrices
R01 = rotZ(th)
R10 = R01.T

R12 = rotX(alph)
R21 = R12.T

R02 = R12*R01
R20 = R02.T

# Positions-----------------------------------------------
##### Arm is at the origin so no need for X0 and Y0
r_Hub   = sym.Matrix([0, 0, 0])
r_Arm_1 = sym.Matrix([La/2, 0, 0])
r_Arm = R10*r_Arm_1

r_Pen_2 = sym.Matrix([0, 0, -Lp/2])
r_Pen = r_Arm*2 + R20*r_Pen_2

#display(r_Arm, r_Pen)

# Velocities-------------------------------------------------
## Linear
dr_Arm = sym.simplify(r_Arm.jacobian(q)*dq)
dr_Pen = sym.simplify(r_Pen.jacobian(q)*dq)
display(dr_Arm,dr_Pen)

## Angular
wArm = sym.Matrix([0, 0, dth])
wPen = R12*wArm + sym.Matrix([dalph, 0, 0])

display(wArm, wPen)

# Energies
## Kinetic energies
Thub = 0
TArm = (0.5*mA*dr_Arm.T*dr_Arm) + (0.5*InA*wArm.T*wArm)
TPen = (0.5*mP*dr_Pen.T*dr_Pen) + (0.5*InP*wPen.T*wPen)
T = sym.Matrix([sym.simplify(TArm + TPen)])
display(T)
#T = sym.Matrix([0.5*mA*(dxArm**2+dyArm**2) + 0.5*mP*(dxPen**2+dyPen**2) + 0.5*InA*dth**2 + 0.5*InP*dalph**2])

## Potential energies
VArm = 0
VHub = 0
VPen = mP*g*r_Pen[2]
V = sym.Matrix([sym.simplify(VArm + VHub + VPen)])
display(V)

In [ ]:
# forces & torques
# tau = sym.symbols(['\\tau'])

# tau_l1 = sym.Matrix([[tau],[0]])

# Qtot = r_Arm.jacobian(q).T*tau_l1

tau = sym.symbols('\\tau')

Qtau = sym.Matrix([[tau],[0]]) # Torque tau acts in the ph1 direction

Qtot = Qtau
# ------------------------------------------------------------------------------------------------------------------
# Mass, Gravity, Coriolis matrices, EOM
LgM = sym.hessian(T, dq)
LgG = V.jacobian(q)

M_col = LgM.reshape(LgM.rows * LgM.cols, 1)
dM = M_col.jacobian(q)*dq
dM = dM.reshape(LgM.rows, LgM.cols)
Tj = T.jacobian(q)
LgC = dM*dq - Tj.T


EOM = LgM*ddq + LgC + LgG.T - Qtot
EOM0 = LgM*ddq #- QGRF

display(EOM, EOM0)

In [ ]:
# Torque
# STEP 5: calculate generalized forces
# control torque
tau = sym.symbols('\\tau')
tau1 = sym.Matrix([[tau],[0]])

Qtau =  # Torque tau acts in the ph1 direction

Qall = Qtau

# put it all together
EOM = Lg1 - Lg3 + Lg4 - Qall.transpose()
display(Qall)

## Lagrange

In [ ]:
# STEP 4: calculate each term of the Lagrange equation
# term 1
Lg1 = sym.zeros(1,len(q))
for i in range(len(q)):
    dT_ddq = sym.diff(T,dq[i]) # get partial of T in dq_i
    Lg1[i] = dT_ddq.jacobian(q)*dq + dT_ddq.jacobian(dq)*ddq #...then get time derivative of that partial

# term 3
Lg3 = T.jacobian(q) # partial of T in q

# term 4
Lg4 = V.jacobian(q) # partial of U in q

#combine
EOM = sym.simplify(Lg1 - Lg3 + Lg4)

display(EOM) #<-this will give you an idea of the magnitude of the faff a decent symbolic package lets you avoid

# Test

In [ ]:
N = 100
h = 0.01
t = np.arange(0,N*h,h)
tau_arr = 50*np.sin(8*np.pi*t)

#initial conditions
th0, alph0, dth0, dalph0 = 0, 0, 0, 0


#parameters
X0val, Y0val, Z0val, l1val, l2val = 0, 0, 0, 1, 1
parameter_values = [(X0,X0val),(Y0,Y0val),(g,9.81),(mA,1),(mP,1),(La,l1val),(Lp,l2val),(InA,0.08),(InP,0.08)]
thar, alphar, dthar, dalphar = [th0], [alph0], [dth0], [dalph0]


for i in range(1,N):
    # substitute parameter values and previous conditions into the EOM 
    past = [(th,thar[i-1]), 
            (alph,alphar[i-1]), 
            (dth,dthar[i-1]), 
            (dalph,dalphar[i-1])]
    forces = [(tau,tau_arr[i-1])]
    EOM_sub = EOM.subs(parameter_values).subs(past).subs(forces)
    
    # solve for the acceleration
    acc = sym.solve(EOM_sub,[ddth, ddalph])
    
    # integrate for the next velocity and position
   
    dalphar.append(float(dalphar[i-1]+h*acc[ddalph]))

    dthar.append(float(dthar[i-1]+h*acc[ddth]))
    
    alphar.append(float(alphar[i-1]+h*dalphar[i]))
    
    thar.append(float(thar[i-1]+h*dthar[i]))

print(thar)
print(alphar)
print(dthar)
print(dalphar)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.animation as ani
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import HTML

# Create the figure and subplots for XY, XZ, YZ planes, and a 3D plot
fig = plt.figure(figsize=(20, 10))
ax_xy = fig.add_subplot(221)  # XY Plane
ax_xz = fig.add_subplot(222)  # XZ Plane
ax_yz = fig.add_subplot(223)  # YZ Plane
ax_3d = fig.add_subplot(224, projection='3d')  # 3D plot

def plot_pendulum(i, th_in, ax_xy, ax_xz, ax_yz, ax_3d, alph_in):  
    # Clear previous plots
    ax_xy.clear()
    ax_xz.clear()
    ax_yz.clear()
    ax_3d.clear()
    
    # Set limits and titles for each axis
    ax_xy.set_xlim([-4, 4])
    ax_xy.set_ylim([-4, 4])
    ax_xy.set_title('Z view - XY Plane')
    ax_xy.set_xlabel('X-axis', color='red')
    ax_xy.set_ylabel('Y-axis', color='blue')
    ax_xy.grid(True)

    ax_xz.set_xlim([-4, 4])
    ax_xz.set_ylim([-4, 4])
    ax_xz.set_title('Y view - XZ Plane')
    ax_xz.set_xlabel('X-axis', color='red')
    ax_xz.set_ylabel('Z-axis', color='green')
    ax_xz.grid(True)
    
    ax_yz.set_xlim([-4, 4])
    ax_yz.set_ylim([-4, 4])
    ax_yz.set_title('X view - YZ Plane')
    ax_yz.set_xlabel('Y-axis', color='blue')
    ax_yz.set_ylabel('Z-axis', color='green')
    ax_yz.grid(True)
    
    ax_3d.set_xlim([-4, 4])
    ax_3d.set_ylim([-4, 4])
    ax_3d.set_zlim([-4, 4])
    ax_3d.set_title('3D View')
    ax_3d.set_xlabel('X-axis', color='red')
    ax_3d.set_ylabel('Y-axis', color='blue')
    ax_3d.set_zlabel('Z-axis', color='green')
    ax_3d.grid(True)

    # Calculate positions for the pendulum
    L1bottomx = X0val + l1val * np.cos(th_in[i])
    L1bottomy = Y0val + l1val * np.sin(th_in[i])
    L1bottomz = Z0val

    L2bottomx = L1bottomx - l2val * np.sin(th_in[i]) * np.sin(alph_in[i])
    L2bottomy = L1bottomy - l2val * np.sin(th_in[i]) * np.cos(alph_in[i])
    L2bottomz = L1bottomz - l2val * np.cos(th_in[i])

    # Plot in XY Plane
    ax_xy.plot([X0val, L1bottomx], [Y0val, L1bottomy], color='xkcd:blue')
    ax_xy.plot([L1bottomx, L2bottomx], [L1bottomy, L2bottomy], color='xkcd:red')
    
    # Plot in XZ Plane
    ax_xz.plot([X0val, L1bottomx], [Z0val, L1bottomz], color='xkcd:blue')
    ax_xz.plot([L1bottomx, L2bottomx], [L1bottomz, L2bottomz], color='xkcd:red')

    # Plot in YZ Plane
    ax_yz.plot([Y0val, L1bottomy], [Z0val, L1bottomz], color='xkcd:blue')
    ax_yz.plot([L1bottomy, L2bottomy], [L1bottomz, L2bottomz], color='xkcd:red')

    # Plot in 3D
    ax_3d.plot([X0val, L1bottomx], [Y0val, L1bottomy], [Z0val, L1bottomz], color='xkcd:blue')
    ax_3d.plot([L1bottomx, L2bottomx], [L1bottomy, L2bottomy], [L1bottomz, L2bottomz], color='xkcd:red')

# Update function for the animation
update = lambda i: plot_pendulum(i, thar, ax_xy, ax_xz, ax_yz, ax_3d, alphar)

# Create animation
animate = ani.FuncAnimation(fig, update, range(N), interval=50, repeat=True)

# Embed the animation in the notebook
HTML(animate.to_html5_video())
